# Ethereum Analytics

<hr/>

## Dataset

https://www.kaggle.com/bigquery/ethereum-blockchain

<hr/>

## Description

### Context

Bitcoin and other cryptocurrencies have captured the imagination of technologists, financiers, and economists. Digital currencies are only one application of the underlying blockchain technology. Like its predecessor, Bitcoin, the Ethereum blockchain can be described as an immutable distributed ledger. However, the set of capabilities have been extended by including a virtual machine that can execute arbitrary code stored on the blockchain known as **Smart Contracts**.

Both Bitcoin and Ethereum are essentially OLTP databases, and provide little in the way of OLAP (analytics) functionality. However the Ethereum dataset is notably distinct from the Bitcoin dataset:

1. The Ethereum blockchain has as its primary unit of value Ether, while the Bitcoin blockchain has Bitcoin. However, the majority of value transfer on the Ethereum blockchain is composed of so-called tokens. Tokens are created and managed by smart contracts.


2. Ether value transfers are precise and direct, resembling accounting ledger debits and credits. This is in contrast to the Bitcoin value transfer mechanism, for which it can be difficult to determine the balance of a given wallet address.


3. Addresses can be not only wallets that hold balances, but can also contain smart contract bytecode that allows the programmatic creation of agreements and automatic triggering of their execution. An aggregate of coordinated smart contracts could be used to build a decentralized autonomous organization.

### Content

The Ethereum blockchain data is available for exploration with BigQuery. All historical data is in the ethereum_blockchain dataset, which updates daily. We have leveraged this BigQuery dataset for our research project.

The Dataset consists of 7 tables which will be analysed throughout the project. The tables are-

1. blocks
2. contracts
3. logs
4. token_transfers
5. tokens
6. traces
7. transactions

<hr/>

## Total number of transactions in the Ethereum network

In [ ]:
query = """
SELECT 
    COUNT(transactions.hash)
FROM
    `bigquery-public-data.crypto_ethereum.transactions` AS transactions
WHERE block_timestamp < '2021-09-01 00:00:00'
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=90)

In [ ]:
print("Total number of transactions in the Ethereum network:", df.iloc[0]["f0_"])

## Average number of transactions per day

In [ ]:
query = """
SELECT 
    COUNT(*) / COUNT(DISTINCT(DATE(block_timestamp)))
FROM
    `bigquery-public-data.crypto_ethereum.transactions` AS transactions
WHERE block_timestamp < '2021-09-01 00:00:00'
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=10)

In [ ]:
print("Average number of transactions per day:", df.iloc[0]["f0_"])

## Number of transactions over time

In [ ]:
query = """
SELECT 
    DATE(block_timestamp) as date, COUNT(*) as number_of_transactions
FROM
    `bigquery-public-data.crypto_ethereum.transactions` AS transactions
WHERE
    block_timestamp < '2021-09-01 00:00:00'
GROUP BY date
ORDER BY date
"""

In [ ]:
from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=10)

In [ ]:
df.head()
df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Number of transactions over time", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Number of transactions", labelpad=20)
sns.lineplot(x="date", y="number_of_transactions", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
df[df["number_of_transactions"] == df["number_of_transactions"].max()]

In [ ]:
df["number_of_transactions"].sum()

## Value of Ether transferred per day

In [ ]:
query = """
SELECT 
    DATE(block_timestamp) AS date, (SUM(value) / power(10,18)) AS total_value 
FROM
    `bigquery-public-data.crypto_ethereum.transactions` AS transactions
WHERE
    block_timestamp < '2021-09-01 00:00:00'
GROUP BY date
ORDER BY date
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=30)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Value of Ether transferred per day", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Ether transferred ", labelpad=20)
sns.lineplot(x="date", y="total_value", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
df[df["total_value"] == df["total_value"].max()]

In [ ]:
df["total_value"].sum()

In [ ]:
df["total_value"].mean()

## Daily gas consumption

In [ ]:
query = """
SELECT 
    DATE(timestamp) as date, SUM(gas_used) as gas_used
FROM 
    `bigquery-public-data.crypto_ethereum.blocks`
WHERE
    timestamp < '2021-09-01 00:00:00'
GROUP BY date
ORDER BY date
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=1)

In [ ]:
df_cleaned = df[1:]
df_cleaned.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Daily gas consumption", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Gas consumed", labelpad=20)
sns.lineplot(x="date", y="gas_used", data=df_cleaned, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
df[df["gas_used"] == df["gas_used"].max()]

In [ ]:
df["gas_used"].sum()

In [ ]:
df["gas_used"].mean()

## Average gas price per day

In [ ]:
query = """
SELECT
    DATE(block_timestamp) as date, avg(gas_price) as average_gas_price
FROM 
    `bigquery-public-data.crypto_ethereum.transactions` 
WHERE 
    receipt_status = 1 AND block_timestamp < '2021-09-01 00:00:00'
GROUP BY date
ORDER BY date
"""

from bq_helper import BigQueryHelper

# This establishes an authenticated session and prepares a reference to the dataset that lives in BigQuery.
bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=29)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Average gas price per day", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Average gas price (in Wei)", labelpad=20)
sns.lineplot(x="date", y="average_gas_price", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
df[df["average_gas_price"] == df["average_gas_price"].max()]

In [ ]:
df["average_gas_price"].sum()

In [ ]:
df["average_gas_price"].mean()

## Active Address Growth per day

In [ ]:
query = """
    SELECT first_transaction AS date, COUNT(*) AS no_of_addresses_making_first_txn
    FROM (
        SELECT from_address AS address, MIN(DATE(block_timestamp)) AS first_transaction
        FROM `bigquery-public-data.crypto_ethereum.transactions`
        WHERE value > 0 AND block_timestamp < '2021-09-01 00:00:00'
        GROUP BY address
        ORDER BY first_transaction
    )
    GROUP BY first_transaction
    ORDER BY date
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=81)

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Active address growth per day", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Number of addresses making first transaction", labelpad=20)
sns.lineplot(x="date", y="no_of_addresses_making_first_txn", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
df["cumsum_addresses"] = df["no_of_addresses_making_first_txn"].cumsum()
df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Active address growth per day", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Cumulative number of addresses making first transaction", labelpad=20)
sns.lineplot(x="date", y="cumsum_addresses", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
query = """
    SELECT first_transaction AS date, COUNT(*) AS no_of_addresses_making_first_txn
    FROM (
        SELECT from_address AS address, MIN(DATE(block_timestamp)) AS first_transaction
        FROM `bigquery-public-data.crypto_ethereum.transactions`
        WHERE block_timestamp < '2021-09-01 00:00:00'
        GROUP BY address
        ORDER BY first_transaction
    )
    GROUP BY first_transaction
    ORDER BY date
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=81)

In [ ]:
df["cumsum_addresses"] = df["no_of_addresses_making_first_txn"].cumsum()
df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Active address growth per day", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Cumulative number of addresses making first transaction", labelpad=20)
sns.lineplot(x="date", y="cumsum_addresses", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

## Total number of contracts

In [ ]:
query = """
SELECT
    COUNT(DISTINCT(address)) as number_of_contracts
FROM
    `bigquery-public-data.crypto_ethereum.contracts` 
WHERE 
    block_timestamp < '2021-09-01 00:00:00'
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=4.5)

In [ ]:
print("Total number of contracts:", df.iloc[0]["number_of_contracts"])

## Number of Smart Contracts created per day

In [ ]:
query = """
SELECT COUNT(DISTINCT(address)) as number_of_contracts, DATE(block_timestamp) as date
FROM `bigquery-public-data.crypto_ethereum.contracts`
WHERE 
    block_timestamp < '2021-09-01 00:00:00'
GROUP BY date
ORDER BY date
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=4.5)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Number of Smart Contracts created per day", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Number of Smart Contracts", labelpad=20)
sns.lineplot(x="date", y="number_of_contracts", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
df["cumsum_number_of_contracts"] = df["number_of_contracts"].cumsum()
df.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Cumulative number of Smart Contracts created per day", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Cumulative number of Smart Contracts", labelpad=20)
sns.lineplot(x="date", y="cumsum_number_of_contracts", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

## Contract creation transactions per day (Directly by users)

In [ ]:
query = """
SELECT COUNT(*) as contracts_created, DATE(block_timestamp) as date
FROM `bigquery-public-data.crypto_ethereum.transactions` 
WHERE to_address IS null AND block_timestamp < '2021-09-01 00:00:00'
GROUP BY date
ORDER BY date
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=62)

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Number of Smart Contracts created per day (Directly by users)", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Number of Smart Contracts", labelpad=20)
sns.lineplot(x="date", y="contracts_created", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
df["cumsum_contracts_created"] = df["contracts_created"].cumsum()
df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.title("Cumulative number of Smart Contracts created per day (Directly by users)", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Cumulative number of Smart Contracts", labelpad=20)
sns.lineplot(x="date", y="cumsum_contracts_created", data=df, color="purple")
plt.gcf().set_dpi(400)
plt.show(fig)

## Daily ether volume for ether transfers, contract calls, and contract creations

In [ ]:
query = """
WITH a AS (
    SELECT DATE(block_timestamp) AS date, SUM(value)/POWER(10,18) AS contract_calls
    FROM `bigquery-public-data.crypto_ethereum.transactions`
    WHERE input != '0x' AND to_address IS NOT null AND block_timestamp < '2021-09-01 00:00:00'
    GROUP BY date
), 
b AS (
    SELECT DATE(block_timestamp) AS date2, SUM(value)/POWER(10,18) AS contract_creation
    FROM `bigquery-public-data.crypto_ethereum.transactions`
    WHERE to_address IS null AND block_timestamp < '2021-09-01 00:00:00'
    GROUP BY date2
), 
c AS (
    SELECT DATE(block_timestamp) AS date3, SUM(value)/POWER(10,18) AS ether_transfers
    FROM `bigquery-public-data.crypto_ethereum.transactions`
    WHERE input = '0x' AND value IS NOT null AND block_timestamp < '2021-09-01 00:00:00'
    GROUP BY date3
)
SELECT contract_calls, ether_transfers, contract_creation, date
FROM a JOIN c ON date=date3 JOIN b ON date=date2
ORDER BY date
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=331)

In [ ]:
df

In [ ]:
total_contract_calls = int(df["contract_calls"].sum())
total_ether_transfers = int(df["ether_transfers"].sum())
total_contract_creations = int(df["contract_creation"].sum())

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np

labels = (str(total_contract_calls)+'\ncontract_calls', str(total_ether_transfers)+'\nether_transfers', str(total_contract_creations)+'\ncontract_creations')
sizes = [total_contract_calls, total_ether_transfers, total_contract_creations]
colors = ["mediumspringgreen", "turquoise", "crimson"]
explode = (0, 0.3, 0.2)  # explode 1st slice

the_grid = GridSpec(2, 2)
plt.figure(1, figsize=(6, 3)) 
plt.subplot(the_grid[0, 1], aspect=2, title='Types of transactions in Ethereum')
plt.rcParams.update({'font.size': 3})
plt.pie(sizes, labels=labels, explode=explode, autopct='%0.1f%%', colors=colors)
plt.gcf().set_dpi(300)
plt.axis('equal')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
# axs.set(yscale="log")
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.rcParams.update({'font.size': 14})
plt.title("Daily ether volume for ether transfers, contract calls, and contract creations", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Daily ether volume", labelpad=20)
sns.lineplot(x="date", y="ether_transfers", data=df, color="slategray", label='ether_transfers')
sns.lineplot(x="date", y="contract_calls", data=df, color="lawngreen", label='contract_calls')
sns.lineplot(x="date", y="contract_creation", data=df, color="purple", label='contract_creations')
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
# Daily volume of ether transferred
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.set(yscale="log")
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.rcParams.update({'font.size': 14})
plt.title("Daily volume of ether transferred (measured in logarithmic scale)", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Ether volume", labelpad=20)
sns.lineplot(x="date", y="ether_transfers", data=df, color="crimson", label='ether_transfers')
sns.lineplot(x="date", y="contract_calls", data=df, color="navy", label='contract_calls')
plt.gcf().set_dpi(400)
plt.show(fig)

## Ether balance distribution

In [ ]:
# KAGGLE NOT ABLE TO PROCESS: LARGE QUERY

# query = """
# WITH value_table AS (
#     SELECT to_address AS address, value AS value
#     FROM `bigquery-public-data.crypto_ethereum.traces`
#     WHERE to_address IS NOT null
#         AND block_timestamp < '2021-09-01 00:00:00'
#         AND status=1 
#         AND (call_type NOT IN ('delegatecall', 'callcode', 'staticcall') OR call_type IS null)
#     UNION ALL
#         SELECT from_address AS address, -value AS value
#         FROM `bigquery-public-data.crypto_ethereum.traces`
#         WHERE from_address IS NOT null
#             AND block_timestamp < '2021-09-01 00:00:00'
#             AND status=1 
#             AND (call_type NOT IN ('delegatecall', 'callcode', 'staticcall') OR call_type IS null)
#     UNION ALL
#         SELECT miner AS address, SUM(CAST(receipt_gas_used AS numeric) * CAST(gas_price AS numeric)) AS value
#         FROM `bigquery-public-data.crypto_ethereum.transactions` AS transactions
#         JOIN `bigquery-public-data.crypto_ethereum.blocks` AS blocks
#         ON blocks.number = transactions.block_number
#         WHERE block_timestamp < '2021-09-01 00:00:00'
#         GROUP BY blocks.miner 
#     UNION ALL
#         SELECT from_address AS address, -(CAST(receipt_gas_used AS numeric) * CAST(gas_price AS numeric)) AS value
#         FROM `bigquery-public-data.crypto_ethereum.transactions`
#         WHERE block_timestamp < '2021-09-01 00:00:00'
# ), 
# a AS (
#     SELECT SUM(value)/POWER(10,18) AS balance, address 
#     FROM value_table
#     GROUP BY address
# ) 
# SELECT balance, row_number() over (
#     ORDER BY balance DESC
# ) AS number_of_addresses FROM a
# ORDER BY balance DESC
# """

# from bq_helper import BigQueryHelper

# bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
# df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=550)

In [ ]:
# df

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# import seaborn as sns

# fig, axs = plt.subplots(figsize=(20, 6))
# axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
# plt.title("Ether balance distribution", y=1.05)
# plt.xlabel("Addresses", labelpad=20)
# plt.ylabel("Ether balance", labelpad=20)
# sns.lineplot(x="number_of_addresses", y="balance", data=df, color="purple")
# plt.gcf().set_dpi(400)
# plt.show(fig)

## Ether Transactions distribution

In [ ]:
# SOME PROBLEM

# query = """
# WITH value_table AS (
#     SELECT to_address AS address, COUNT(DISTINCT(transactions.hash)) AS tx
#     FROM `bigquery-public-data.crypto_ethereum.transactions` AS transactions
#     WHERE to_address IS NOT null
#         AND block_timestamp < '2021-09-01 00:00:00'
#     GROUP BY address
#     UNION ALL
#         SELECT from_address AS address, COUNT(DISTINCT(transactions.hash)) as tx
#         FROM `bigquery-public-data.crypto_ethereum.transactions` AS transactions
#         WHERE from_address IS NOT null
#             AND block_timestamp < '2021-09-01 00:00:00'
#         GROUP BY address
# ),
# a AS (
#     SELECT SUM(tx) AS transactions, address
#     FROM value_table
#     GROUP BY address
# ),
# b AS (
#     SELECT transactions, row_number() over(
#         ORDER BY transactions DESC
#     ) AS address_rank FROM a
# ),
# c AS (
#     SELECT transactions, floor(log(address_rank)*100) AS log_address_rank, address_rank
#     FROM b
# )
# SELECT avg(transactions) AS transactions_avg, log_address_rank, COUNT(address_rank) AS number_of_addresses
# FROM c
# GROUP BY log_address_rank
# ORDER BY transactions_avg DESC
# """

# from bq_helper import BigQueryHelper

# bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
# df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=1)

In [ ]:
# df

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# import seaborn as sns

# fig, axs = plt.subplots(figsize=(20, 6))
# axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
# axs.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
# plt.title("Ether transactions distribution", y=1.05)
# plt.xlabel("Number of Sender Addresses", labelpad=20)
# plt.ylabel("Ether transactions", labelpad=20)
# sns.lineplot(x="number_of_addresses", y="transactions_avg", data=df, color="purple")
# plt.axis([0, 40, 0, 8000000]) # Zoom in
# plt.gcf().set_dpi(400)
# plt.show(fig)

## Top 10 addresses measured by balance (value of ether)

In [ ]:
query = """
WITH value_table AS (
    SELECT to_address AS address, value AS value
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE to_address IS NOT null
    AND block_timestamp < '2021-09-01 00:00:00'
    AND status=1
    AND (call_type NOT IN ('delegatecall', 'callcode', 'staticcall') OR call_type IS null)
    
    UNION ALL
    
    SELECT from_address AS address, -value AS value
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE from_address IS NOT null
    AND block_timestamp < '2021-09-01 00:00:00'
    AND status=1
    AND (call_type NOT IN ('delegatecall', 'callcode', 'staticcall') OR call_type IS null)
    
    UNION ALL
    
    SELECT miner as address, SUM(CAST(receipt_gas_used AS NUMERIC) * CAST(gas_price AS NUMERIC)) AS value
    FROM `bigquery-public-data.crypto_ethereum.transactions` AS transactions
    JOIN `bigquery-public-data.crypto_ethereum.blocks` AS blocks
    ON blocks.number = transactions.block_number
    WHERE block_timestamp < '2021-09-01 00:00:00'
    GROUP BY blocks.miner
    
    UNION ALL
    
    SELECT from_address as address, -(CAST(receipt_gas_used AS NUMERIC) * CAST(gas_price AS NUMERIC)) AS value
    FROM  `bigquery-public-data.crypto_ethereum.transactions`
    WHERE block_timestamp < '2021-09-01 00:00:00'
)
SELECT address, FLOOR(SUM(value) / power(10,18)) AS balance
FROM value_table
GROUP BY address
ORDER BY balance DESC
LIMIT 10
"""

from bq_helper import BigQueryHelper

# This establishes an authenticated session and prepares a reference to the dataset that lives in BigQuery.
bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=21)

In [ ]:
df

## Top 10 addresses measured by number of sent transactions

In [ ]:
query = """
WITH value_table AS (
    SELECT to_address AS address, value AS value
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE to_address IS NOT null
    AND block_timestamp < '2021-09-01 00:00:00'
    AND status=1
    AND (call_type NOT IN ('delegatecall', 'callcode', 'staticcall') OR call_type IS null)
    
    UNION ALL
    
    SELECT from_address AS address, -value AS value
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE from_address IS NOT null
    AND block_timestamp < '2021-09-01 00:00:00'
    AND status=1
    AND (call_type NOT IN ('delegatecall', 'callcode', 'staticcall') OR call_type IS null)
    
    UNION ALL
    
    SELECT miner as address, SUM(CAST(receipt_gas_used AS NUMERIC) * CAST(gas_price AS NUMERIC)) AS value
    FROM `bigquery-public-data.crypto_ethereum.transactions` AS transactions
    JOIN `bigquery-public-data.crypto_ethereum.blocks` AS blocks
    ON blocks.number = transactions.block_number
    WHERE block_timestamp < '2021-09-01 00:00:00'
    GROUP BY blocks.miner
    
    UNION ALL
    
    SELECT from_address as address, -(CAST(receipt_gas_used AS NUMERIC) * CAST(gas_price AS NUMERIC)) AS value
    FROM  `bigquery-public-data.crypto_ethereum.transactions`
    WHERE block_timestamp < '2021-09-01 00:00:00'
),
a AS (
    SELECT SUM(value)/POWER(10,18) AS balance, address
    FROM value_table
    GROUP BY address
    ORDER BY balance DESC
),
b AS (
    SELECT to_address, COUNT(transactions.hash) AS tx_recipient
    FROM  `bigquery-public-data.crypto_ethereum.transactions` AS transactions
    WHERE block_timestamp < '2021-09-01 00:00:00'
    GROUP BY to_address
), 
c AS (
    SELECT from_address, COUNT(transactions.hash) AS tx_sender
    FROM  `bigquery-public-data.crypto_ethereum.transactions` AS transactions
    WHERE block_timestamp < '2021-09-01 00:00:00'
    GROUP BY from_address
)
SELECT from_address, tx_sender, balance
FROM c LEFT JOIN a ON (a.address = c.from_address)
ORDER BY tx_sender DESC
LIMIT 10
"""

from bq_helper import BigQueryHelper

# This establishes an authenticated session and prepares a reference to the dataset that lives in BigQuery.
bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=63)

In [ ]:
df

## Top 10 addresses measured by number of received transactions

In [ ]:
query = """
WITH value_table AS (
    SELECT to_address AS address, value AS value
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE to_address IS NOT null
    AND block_timestamp < '2021-09-01 00:00:00'
    AND status=1
    AND (call_type NOT IN ('delegatecall', 'callcode', 'staticcall') OR call_type IS null)
    
    UNION ALL
    
    SELECT from_address AS address, -value AS value
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE from_address IS NOT null
    AND block_timestamp < '2021-09-01 00:00:00'
    AND status=1
    AND (call_type NOT IN ('delegatecall', 'callcode', 'staticcall') OR call_type IS null)
    
    UNION ALL
    
    SELECT miner as address, SUM(CAST(receipt_gas_used AS NUMERIC) * CAST(gas_price AS NUMERIC)) AS value
    FROM `bigquery-public-data.crypto_ethereum.transactions` AS transactions
    JOIN `bigquery-public-data.crypto_ethereum.blocks` AS blocks
    ON blocks.number = transactions.block_number
    WHERE block_timestamp < '2021-09-01 00:00:00'
    GROUP BY blocks.miner
    
    UNION ALL
    
    SELECT from_address as address, -(CAST(receipt_gas_used AS NUMERIC) * CAST(gas_price AS NUMERIC)) AS value
    FROM  `bigquery-public-data.crypto_ethereum.transactions`
    WHERE block_timestamp < '2021-09-01 00:00:00'
),
a AS (
    SELECT SUM(value)/POWER(10,18) AS balance, address
    FROM value_table
    GROUP BY address
    ORDER BY balance DESC
),
b AS (
    SELECT to_address, COUNT(transactions.hash) AS tx_recipient
    FROM  `bigquery-public-data.crypto_ethereum.transactions` AS transactions
    WHERE block_timestamp < '2021-09-01 00:00:00'
    GROUP BY to_address
), 
c AS (
    SELECT from_address, COUNT(transactions.hash) AS tx_sender
    FROM  `bigquery-public-data.crypto_ethereum.transactions` AS transactions
    WHERE block_timestamp < '2021-09-01 00:00:00'
    GROUP BY from_address
)
SELECT to_address, tx_recipient, balance
FROM b LEFT JOIN a ON (a.address = b.to_address)
ORDER BY tx_recipient DESC
LIMIT 10
"""

from bq_helper import BigQueryHelper

# This establishes an authenticated session and prepares a reference to the dataset that lives in BigQuery.
bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=680)

In [ ]:
df

##  Cumulative number of contracts created by users and of contracts created by other contracts

In [1]:
query = """
WITH a AS (
    SELECT DATE(block_timestamp) AS date, COUNT(*) AS contracts_creation
    FROM  `bigquery-public-data.crypto_ethereum.traces` AS traces
    WHERE block_timestamp < '2021-09-01 00:00:00'
        AND trace_type = 'create'
        AND trace_address IS null
        GROUP BY date
),
b AS (
    SELECT date, SUM(contracts_creation)
    OVER (ORDER BY date) AS ccc, LEAD(date, 1) OVER (ORDER BY date) AS next_date
    FROM a
    ORDER BY date
),
calendar AS (
    SELECT date
    FROM UNNEST(generate_date_array('2015-07-30', '2021-08-31')) AS date
),
c AS (
    SELECT calendar.date, ccc
    FROM b JOIN calendar ON b.date <= calendar.date
    AND calendar.date < b.next_date
    ORDER BY calendar.date
),
d AS (
    SELECT date(block_timestamp) AS date1, COUNT(*) AS contracts_creation1
    FROM  `bigquery-public-data.crypto_ethereum.traces` AS traces
    WHERE block_timestamp < '2021-09-01 00:00:00'
        AND trace_type = 'create'
        AND trace_address IS NOT null
        GROUP BY date1
),
e AS (
    SELECT date1, SUM(contracts_creation1)
    OVER (ORDER BY date1) AS ccc1, LEAD(date1, 1) OVER (ORDER BY date1) AS next_date1
    FROM d
    ORDER BY date1
),
calendar1 AS (
    SELECT date1
    FROM UNNEST(generate_date_array('2015-07-30', '2021-08-31')) AS date1
),
f AS (
    SELECT calendar1.date1, ccc1
    FROM e JOIN calendar1 ON e.date1 <= calendar1.date1
        AND calendar1.date1 < e.next_date1
    ORDER BY calendar1.date1
)
SELECT date1, date, f.ccc1 as cumulative_contract_creation_by_contracts, c.ccc as cumulative_contract_creation_by_users
FROM c JOIN f ON f.date1 = c.date
ORDER BY f.date1
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=62)

Using Kaggle's public dataset BigQuery integration.


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.rcParams.update({'font.size': 14})
plt.title("Cumulative number of contracts created by users and of contracts created by other contracts", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Number of contracts", labelpad=20)
sns.lineplot(x="date", y="cumulative_contract_creation_by_contracts", data=df, color="crimson", label='cumulative_contract_creation_by_contracts')
sns.lineplot(x="date", y="cumulative_contract_creation_by_users", data=df, color="navy", label='cumulative_contract_creation_by_users')
plt.gcf().set_dpi(400)
plt.show(fig)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

fig, axs = plt.subplots(figsize=(20, 6))
axs.set(yscale="log")
axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.rcParams.update({'font.size': 14})
plt.title("Cumulative number of contracts created by users and of contracts created by other contracts", y=1.05)
plt.xlabel("Date", labelpad=20)
plt.ylabel("Number of contracts", labelpad=20)
sns.lineplot(x="date", y="cumulative_contract_creation_by_contracts", data=df, color="crimson", label='cumulative_contract_creation_by_contracts')
sns.lineplot(x="date", y="cumulative_contract_creation_by_users", data=df, color="navy", label='cumulative_contract_creation_by_users')
plt.gcf().set_dpi(400)
plt.show(fig)

## Number of unique creators of user-created contracts

In [ ]:
query = """
SELECT COUNT(DISTINCT(from_address)) AS unique_creators
FROM  `bigquery-public-data.crypto_ethereum.traces` AS traces
WHERE block_timestamp < '2021-09-01 00:00:00'
    AND trace_type='create'
    AND trace_address IS null
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=340)

In [ ]:
df

## Total number of bytecodes of user-created contracts

In [ ]:
query = """
SELECT COUNT(output) AS unique_bytecodes
FROM  `bigquery-public-data.crypto_ethereum.traces` AS traces
WHERE block_timestamp < '2021-09-01 00:00:00'
    AND trace_type='create'
    AND trace_address IS null
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=340)
df

## Number of unique bytecodes of user-created contracts

In [ ]:
query = """
SELECT COUNT(DISTINCT(output)) AS unique_bytecodes
FROM  `bigquery-public-data.crypto_ethereum.traces` AS traces
WHERE block_timestamp < '2021-09-01 00:00:00'
    AND trace_type='create'
    AND trace_address IS null
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=340)

In [ ]:
df

## Number of unique creators of contract-created contracts

In [ ]:
query = """
SELECT COUNT(DISTINCT(from_address)) AS unique_creators
FROM  `bigquery-public-data.crypto_ethereum.traces` AS traces
WHERE block_timestamp < '2021-09-01 00:00:00'
    AND trace_type='create'
    AND trace_address IS NOT null
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=340)

In [ ]:
df

## Total number of bytecodes of contract-created contracts

In [ ]:
query = """
SELECT COUNT(output) AS unique_bytecodes
FROM  `bigquery-public-data.crypto_ethereum.traces` AS traces
WHERE block_timestamp < '2021-09-01 00:00:00'
    AND trace_type='create'
    AND trace_address IS NOT null
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=340)
df

## Number of unique bytecodes of contract-created contracts

In [2]:
query = """
SELECT COUNT(DISTINCT(output)) AS unique_bytecodes
FROM  `bigquery-public-data.crypto_ethereum.traces` AS traces
WHERE block_timestamp < '2021-09-01 00:00:00'
    AND trace_type='create'
    AND trace_address IS NOT null
"""

from bq_helper import BigQueryHelper

bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=340)

Using Kaggle's public dataset BigQuery integration.


RetryError: Deadline of 120.0s exceeded while calling functools.partial(functools.partial(<bound method _DataProxyConnection.api_request of <kaggle_gcp._DataProxyConnection object at 0x7fdc86e80810>>, method='POST', path='/projects/kaggle-161607/jobs', data={'jobReference': {'jobId': '6213723c-11b6-49db-a6e2-aa8c47134f9d', 'projectId': 'kaggle-161607'}, 'configuration': {'query': {'useLegacySql': False, 'query': "\nSELECT COUNT(DISTINCT(output)) AS unique_bytecodes\nFROM  `bigquery-public-data.crypto_ethereum.traces` AS traces\nWHERE block_timestamp < '2021-09-01 00:00:00'\n    AND trace_type='create'\n    AND trace_address IS NOT null\n"}, 'dryRun': True}}, timeout=None)), last exception: 429 POST https://dp.kaggle.net/bigquery/v2/projects/kaggle-161607/jobs?prettyPrint=false: Quota exceeded

In [ ]:
df

## What are the 10 most popular Ethereum collectibles (ERC721 contracts), by number of transactions?

In [ ]:
query = """
    SELECT contracts.address, COUNT(1) AS tx_count
    FROM `bigquery-public-data.crypto_ethereum.contracts` AS contracts
    JOIN `bigquery-public-data.crypto_ethereum.transactions` AS transactions ON (transactions.to_address = contracts.address)
    WHERE contracts.is_erc721 = TRUE AND contracts.block_timestamp < '2021-09-01 00:00:00'
    GROUP BY contracts.address
    ORDER BY tx_count DESC
    LIMIT 10
"""

from bq_helper import BigQueryHelper

# This establishes an authenticated session and prepares a reference to the dataset that lives in BigQuery.
bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=55)
df

## What are the 10 most popular Ethereum tokens (ERC20 contracts), by number of transactions?

In [ ]:
query = """
    SELECT contracts.address, COUNT(1) AS tx_count
    FROM `bigquery-public-data.crypto_ethereum.contracts` AS contracts
    JOIN `bigquery-public-data.crypto_ethereum.transactions` AS transactions ON (transactions.to_address = contracts.address)
    WHERE contracts.is_erc20 = TRUE AND contracts.block_timestamp < '2021-09-01 00:00:00'
    GROUP BY contracts.address
    ORDER BY tx_count DESC
    LIMIT 10
"""

from bq_helper import BigQueryHelper

# This establishes an authenticated session and prepares a reference to the dataset that lives in BigQuery.
bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=55)
df

# ------------------------ Gas usage distribution ------------------------

In [ ]:
# from google.cloud import bigquery
# import pandas as pd

# query = """
#     select format_timestamp('%Y-%m-%d' ,TIMESTAMP_TRUNC(block_timestamp, WEEK) ) as week,
#     round(log10(receipt_gas_used), 1) as log10_gas_used, count(*) as ntx 
#     from `bigquery-public-data.crypto_ethereum.transactions`
#     group by week, log10_gas_used
#     order by week
# """

# bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
# df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=139)
# df

In [ ]:
# df_pivot = pd.pivot_table(data=df, values='ntx', columns='week', index='log10_gas_used')
# df_pivot.head()

In [ ]:
# from matplotlib import pylab as plt
# %matplotlib inline
# from matplotlib.pylab import rcParams
# import numpy as np
# import seaborn as sns

# fig = plt.figure(figsize=(12,6))
# ax=sns.heatmap(df_pivot, cmap='Reds', cbar_kws={'label': '# of txns'});
# ax.invert_yaxis()

In [ ]:
# df['log10_ntx'] = np.log10( df['ntx'])
# df_pivot = pd.pivot_table(data=df, values='log10_ntx', columns='week', index='log10_gas_used')

# fig = plt.figure(figsize=(12,6))
# ax=sns.heatmap(df_pivot, cmap='Blues', cbar_kws={'label': 'Log10(# of txns)'});
# ax.invert_yaxis()

## Ether supply growth rate

In [ ]:
# query = """
# WITH a as (
#     SELECT 
#         DATE(block_timestamp) as date, SUM(value) as value
#     FROM
#         `bigquery-public-data.crypto_ethereum.traces`
#     WHERE trace_type in ('genesis', 'reward')
#         AND block_timestamp < '2021-09-01 00:00:00'
#     GROUP BY date
#     ORDER BY date
# )
# SELECT date, value / POWER(10, 18) AS supply FROM a
# """

# from bq_helper import BigQueryHelper

# bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
# df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=108)
# df

In [ ]:
# df_cleaned = df[1:]
# df_cleaned.head()

In [ ]:
# # Daily volume of ether transferred
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# import seaborn as sns

# fig, axs = plt.subplots(figsize=(20, 6))
# # axs.set(yscale="log")
# axs.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
# plt.rcParams.update({'font.size': 14})
# plt.title("Ether supply growth rate", y=1.05)
# plt.xlabel("Date", labelpad=20)
# plt.ylabel("Supply", labelpad=20)
# sns.lineplot(x="date", y="supply", data=df_cleaned, color="purple", label='supply')
# plt.gcf().set_dpi(400)
# plt.show(fig)

# <----------------- Reverted Transactions over time ---------------->

In [ ]:
# rtx_query = """
# WITH reverted_transactions_in_block as ( 
#     SELECT
#       block_timestamp as timestamp,
#       count(*) as count
#     FROM
#       `bigquery-public-data.crypto_ethereum.transactions`
#     WHERE
#       block_number > 4370000 -- first Byzantium block
#       AND
#       receipt_status = 0 -- reverted transaction indicator
#       AND block_timestamp >= '2021-01-01 00:00:00'
#       AND block_timestamp < '2021-08-31 00:00:00'
#     GROUP BY
#       1
# )

# SELECT
#     *
# FROM
#     reverted_transactions_in_block
# ORDER BY
#     1 asc
# """

# bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
# df_rtx = bq_assistant.query_to_pandas_safe(rtx_query, max_gb_scanned=7)

In [ ]:
# df_rtx['date'] = pd.to_datetime(df_rtx.timestamp)
# agg_rtx = df_rtx.resample('M', on='date').sum()

In [ ]:
# agg_rtx.plot(y='count', title='Reverted Transactions on Ethereum (monthly totals)', figsize=(16, 9))

# *Top 10 highest txns fees*

In [ ]:
# # This table show us the most extreme cases for regular contract calls in ETH 
# query = """
#         SELECT `hash`, `value`, `gas_price` as `gas_price_in_wei`, `receipt_gas_used`, `receipt_cumulative_gas_used`, `gas_price` / 1000000000000000000 * `receipt_gas_used` as fee_in_ether, `block_timestamp`
#         FROM `bigquery-public-data.crypto_ethereum.transactions`
#         ORDER BY `receipt_gas_used` DESC
#         LIMIT 10
#         """
# bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
# df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=139)
# df

In [ ]:
# # This table show us the most extreme cases for regular transfers in ETH

# query = """
#         SELECT `hash`,`value`, `gas_price` as `gas_price_in_wei`, `receipt_gas_used`, `receipt_cumulative_gas_used`, `gas_price` / 1000000000000000000 * `receipt_gas_used` as fee_in_ether, `block_timestamp`
#         FROM `bigquery-public-data.crypto_ethereum.transactions`
#         ORDER BY `gas_price` DESC
#         LIMIT 10
#         """
# bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
# df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=139)
# df

In [ ]:
# # This table show us the most extreme cases for contract calls that also transfer value in ETH
# # notice that "up_front_cost = gas_price * gas_limit + value" and here we are using gas_used instead of gas_limit

# query = """
#         SELECT `hash`, `value`, `gas_price` as `gas_price_in_wei`, `receipt_gas_used`, `receipt_cumulative_gas_used`, `gas_price` / 1000000000000000000 * `receipt_gas_used` as fee_in_ether, (`gas_price` / 1000000000000000000 * `receipt_gas_used`) + `value` / 1000000000000000000 as up_front_cost_aprox_in_ether, `block_timestamp`
#         FROM `bigquery-public-data.crypto_ethereum.transactions`
#         WHERE `receipt_gas_used` > 21000
#         ORDER BY `value` DESC
#         LIMIT 10
#         """
# bq_assistant = BigQueryHelper("bigquery-public-data", "crypto_ethereum")
# df = bq_assistant.query_to_pandas_safe(query, max_gb_scanned=139)
# df